In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import time
from pyspark.sql.types import *
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from pyspark.sql.types import *
from pyspark.sql import functions 
from tensorflow.keras import layers
import tensorflow as tf
from keras.optimizers import Adam

2024-09-26 05:43:06.534753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
customSchema = StructType([StructField("age", FloatType(), True), 
                          StructField("job", FloatType(), True), 
                          StructField("marital", FloatType(), True), 
                          StructField("education", FloatType(), True),
                          StructField("default", FloatType(), True),
                          StructField("balance", FloatType(), True),
                          StructField("housing", FloatType(), True),
                          StructField("loan", FloatType(), True),
                          StructField("contact", FloatType(), True),
                          StructField("day_of_week", FloatType(), True),
                          StructField("month", FloatType(), True),
                          StructField("duration", FloatType(), True),
                          StructField("campaign", FloatType(), True),
                          StructField("pdays", FloatType(), True),
                          StructField("previous", FloatType(), True),
                          StructField("poutcome", FloatType(), True),
                          StructField("y", FloatType(), True)])

df = spark.read.load('hdfs://localhost:9000/user1/Bank-Pyspark1.csv', format="csv", header="true", sep=',', schema=customSchema)

In [3]:
type(df)

pyspark.sql.dataframe.DataFrame

In [4]:
df.printSchema()

root
 |-- age: float (nullable = true)
 |-- job: float (nullable = true)
 |-- marital: float (nullable = true)
 |-- education: float (nullable = true)
 |-- default: float (nullable = true)
 |-- balance: float (nullable = true)
 |-- housing: float (nullable = true)
 |-- loan: float (nullable = true)
 |-- contact: float (nullable = true)
 |-- day_of_week: float (nullable = true)
 |-- month: float (nullable = true)
 |-- duration: float (nullable = true)
 |-- campaign: float (nullable = true)
 |-- pdays: float (nullable = true)
 |-- previous: float (nullable = true)
 |-- poutcome: float (nullable = true)
 |-- y: float (nullable = true)



In [5]:
df.show(20)

+----------+----------+-------+----------+-------+-----------+-------+----+-------+-----------+----------+-----------+--------+-----+--------+--------+---+
|       age|       job|marital| education|default|    balance|housing|loan|contact|day_of_week|     month|   duration|campaign|pdays|previous|poutcome|  y|
+----------+----------+-------+----------+-------+-----------+-------+----+-------+-----------+----------+-----------+--------+-----+--------+--------+---+
| 0.5194805|0.36363637|    0.5| 0.6666667|    0.0| 0.09225936|    1.0| 0.0|    1.0| 0.13333334|0.72727275|0.053070355|     0.0|  0.0|     0.0|     1.0|0.0|
|0.33766234| 0.8181818|    1.0|0.33333334|    0.0| 0.07306666|    1.0| 0.0|    1.0| 0.13333334|0.72727275|0.030703537|     0.0|  0.0|     0.0|     1.0|0.0|
|0.19480519|0.18181819|    0.5|0.33333334|    0.0| 0.07282153|    1.0| 1.0|    1.0| 0.13333334|0.72727275|0.015453436|     0.0|  0.0|     0.0|     1.0|0.0|
|0.37662336|0.09090909|    0.5|       1.0|    0.0| 0.08647613|  

In [23]:
# create model
model = Sequential()
from keras.regularizers import l1
model.add(Dense(500, input_dim = 16, activation='relu' , kernel_regularizer=l1(0.001))) # 500 neurons
model.add(Dense(1, activation='sigmoid')) # 1 output neuron

In [24]:
# Compile model
X_columns = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day_of_week', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
Y_column = 'y'  # Replace with the actual label column name

# Extract X and Y
X_spark = df.select(X_columns)
Y_spark = df.select(Y_column)

# # Convert to NumPy arrays
X = np.array(X_spark.collect())  # Collecting to local NumPy array
Y = np.array(Y_spark.collect()).flatten()  # Flatten Y since it's a single column
X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32) 

optimizer = Adam(clipnorm=10)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# # Fit the model
model.fit(X, Y, epochs=150, batch_size=128,verbose =2)# 150 epoch, 10 batch size, verbose = 2

Epoch 1/150
230/230 - 4s - 16ms/step - accuracy: 0.9450 - loss: nan
Epoch 2/150
230/230 - 2s - 8ms/step - accuracy: 0.9499 - loss: nan
Epoch 3/150
230/230 - 2s - 9ms/step - accuracy: 0.9499 - loss: nan
Epoch 4/150
230/230 - 2s - 8ms/step - accuracy: 0.9499 - loss: nan
Epoch 5/150
230/230 - 1s - 6ms/step - accuracy: 0.9499 - loss: nan
Epoch 6/150
230/230 - 1s - 5ms/step - accuracy: 0.9499 - loss: nan
Epoch 7/150
230/230 - 1s - 4ms/step - accuracy: 0.9499 - loss: nan
Epoch 8/150
230/230 - 1s - 6ms/step - accuracy: 0.9499 - loss: nan
Epoch 9/150
230/230 - 1s - 6ms/step - accuracy: 0.9499 - loss: nan
Epoch 10/150
230/230 - 1s - 6ms/step - accuracy: 0.9499 - loss: nan
Epoch 11/150
230/230 - 1s - 4ms/step - accuracy: 0.9499 - loss: nan
Epoch 12/150
230/230 - 1s - 4ms/step - accuracy: 0.9499 - loss: nan
Epoch 13/150
230/230 - 1s - 3ms/step - accuracy: 0.9499 - loss: nan
Epoch 14/150
230/230 - 1s - 4ms/step - accuracy: 0.9499 - loss: nan
Epoch 15/150
230/230 - 1s - 6ms/step - accuracy: 0.9499 

In [20]:
predictions = model.predict(X)    # predicting Y only using X
print(predictions)
actual = Y
rounded = [round(x[0]) for x in predictions]
predicted = rounded

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show() 

917/917 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step
[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]


ValueError: cannot convert float NaN to integer

In [ ]:
# summarize the first 5 cases
for i in range(5):
	print(str(rounded[i]) + '=>(expected '+ str(Y[i]) +')')